In [7]:
import numpy as np
import math

# Here you will find the Newton method to find the 'D' component for coding

## CHANGE THE VALUE OF Pb IN THE CELL BELOW :

In [8]:
Pb = 1e-6

In [9]:
def compute_D():
    
    '''Here we use the 3 terms approximations.
    Il faut faire attention : Le 1 term approximation n'est valable SEULEMENT SI la proba d'erreur est basse.
    Donc pour Pb ~ 10^-5 il faut commencer à prendre au moins 2 termes.

    Ici avec 3 termes on est laaarge.'''
    
    
    def f(D):
        global Pb
        return 0.5*(36*D**10 + 211*D**12 + 1404*D**14) - Pb
 
    def df(D):
        return 0.5*(360*D**9 + 12*211*D**11 + 14*1404*D**13)
    
    def dx(f, x):
        return abs(0-f(x))

    def newtons_method(f, df, guess, e):
        delta = dx(f, guess)
        while delta > e:
            guess = guess - f(guess)/df(guess)
            delta = dx(f, guess)
        print ('D = ', guess)
        
    
    basic_guess = 0.2
    eps = 1e-15;
    
    return newtons_method(f, df, basic_guess, eps)
    

In [10]:
compute_D()

D =  0.1840822315060514


---------
# In the cells below you will find all the code to compute Fractional Throughput


## First, let's copy paste useful code from link_analysis.ipynb

In [11]:
EARTH_RADIUS = 6378*1000 # m
GEO_RADIUS = 42174*1000 # m, distance between geostationary satellite and center of the EARTH
GEO_ALTITUDE = GEO_RADIUS - EARTH_RADIUS

LEO_ALTITUDE = 1000*1000 # m
LEO_RADIUS = LEO_ALTITUDE + EARTH_RADIUS

'''Let's define utility functions'''

def to_deg(rad):
    return rad*180/math.pi

def to_rad(deg):
    return deg*math.pi/180

def to_dB(decimal):
    return 10*math.log10(decimal)

def to_decimal(dB):
    return 10**(dB/10)

def cos_deg(deg):
    return math.cos(to_rad(deg))

def sin_deg(deg):
    return math.sin(to_rad(deg))

def get_alpha_beta(elevation_angle, GEO=True):
    """
    Compute the angle between the (satellite-earth center) line and the (satellite-earth surface) line.
    The latter has an elevation_angle given with respect to the earth tangent.
    All angles abre computed in degrees
    """
    
    radius = GEO_RADIUS if GEO else LEO_RADIUS
    
    alpha = to_deg(math.asin(EARTH_RADIUS * sin_deg(90. + elevation_angle) / radius))
    beta = 180. - alpha - 90. - elevation_angle
    
    return alpha, beta

def get_range(elevation_angle, GEO=True):
    '''
    Computes the range [meters] of a satellite with a given elevation angle.
    It is possible to define a sat_altitude in case the satellite is not geostationary'''
    
    alti = GEO_ALTITUDE if GEO else LEO_ALTITUDE
    
    alpha, beta = get_alpha_beta(elevation_angle, GEO=GEO)
    return math.sqrt( (alti+EARTH_RADIUS)**2 + EARTH_RADIUS**2 - 2*EARTH_RADIUS*(EARTH_RADIUS+alti) * cos_deg(180-(90+elevation_angle)-alpha)) 

## Be careful ! As explained in the function description below, all the values need to be provided in *bits*. 

## You need to do all the convertions before.
### byte = 8 bits
### kbit = 1024 bits
### Mbit = 1024.1024 bits

In [14]:
def frac_throughput(over_head, BER, W_size_bits, C, MTU_size_bits, process_delay, elevation_angle, GEO=True):
    """
    This function computes the fractional throughput of the link.
    A special attention must be given to the units of the arguments.
    
    - over_head : number between 0 and 1 representing the percentage of overhead (20 % overhead => over_head=0.2)
    - BER : bit error rate
    - W_size_bits : the TCP window size in BITS (**NOT** in terms of MTU)
    - MTU_size_bits : the size of the MTU in BITS
    - C : the controlled flow rate in ** BITS/SECOND **
    - process_delay : duration in seconds that represents the TOTAL PROCESSING delay I_process. It will be added to 
                       I_UpDown (itself computed inside the function using the range of the satellite) to produce the 
                       full delay.
    - elevation_angle : elevation angle of the orbit of the satellite.
    - GEO : a boolean which is 'True' when considering a GEO orbit and 'False' when considering a LEO orbit."""
    
    _1_minus_OH = 1-over_head
    _1_minus_L = (1-BER)**W_size_bits
    
    max_range = get_range(elevation_angle, GEO=GEO) # meters
    
    W_size_MTU = W_size_bits / MTU_size_bits
    print("W = %d [MTU]" % W_size_MTU)
    
    I_tot = (2.*max_range/3e8) + process_delay
    print("Total delay = %.4f seconds" % I_tot)
    
    band_delay_prod = 2*C*I_tot / MTU_size_bits
    print("Bandwidth delay product = %.2f" % band_delay_prod)
    
    result = _1_minus_OH * _1_minus_L * W_size_MTU / (1+band_delay_prod)
    print("Fractional throughput = %.2f %%" % (result*100))
    

In [15]:
frac_throughput(over_head=0.2, BER=0, W_size_bits=8*65536, MTU_size_bits=8*1024, C=1.5*1024*1024, process_delay=0., elevation_angle=0.)

W = 64 [MTU]
Total delay = 0.2779 seconds
Bandwidth delay product = 106.72
Fractional throughput = 47.53 %


---------
# In the cells below you will find all the code to compute Rayleigh's availability


In [35]:
def compute_availability(positive_dB_loss_to_be_handled):
    result = np.exp(-to_decimal(-positive_dB_loss_to_be_handled))
    
    print("The Rayleigh channel can handle at least a %d dB loss.\nThe availability is thus %.3f %%" % (positive_dB_loss_to_be_handled, result))

In [36]:
compute_availability(6)

The Rayleigh channel can handle at least a 6 dB loss.
The availability is thus 0.778 %


In [45]:
def compute_req_margin(availability):
    
    
    if availability < 0. or availability > 1.:
        raise ValueError("The value of the 'availability' parameter must be a double between 0 and 1 (availability is a percentage).")
    
    result = -to_dB(-np.log(availability))
    
    print("We want to have an availability of %.1f %%.\nThus we need a margin of %.2f dB" % (availability*100, result))

In [47]:
compute_req_margin(0.98)

We want to have an availability of 98.0 %.
Thus we need a margin of 16.95 dB


---------
# In the cells below you will find all the code to compute the fade duration


In [65]:
def compute_fade_duration(fade_depth_dB, fd):
    rho = to_decimal(-fade_depth_dB)
    
    result = (np.exp(rho*rho)-1) / (rho*fd*np.sqrt(2*np.pi))
    
    print("The fade duration is %.f microseconds" % (result*pow(10,6)))
    
    

In [66]:
compute_fade_duration(10., 1000.)

The fade duration is 40 microseconds
